# 프로젝트
### 얼굴을 인식하여 캐릭터 씌우기

Face Detection vs Face Recognition

패키지 설치
> ! pip install mediapipe

In [4]:
# ! pip install mediapipe
# ! pip install matplotlib

import mediapipe as mp

In [17]:
import cv2
import mediapipe as mp

# 대상 이미지(3채널 이미지), x, y 좌표, width, height, 덮어씌울 이미지(4 채널 이미지)
def overlay(image, x, y, w, h, overlay_image):
    alpha = overlay_image[:, :, 3] # BGRA
    mask_image = alpha / 255 # 0 ~ 255 => 0 ~ 1(1 : 불투명, 0 : 완전 투명)

    for c in range(0, 3): # Channel BGR
        image[y - h : y + h, x - w : x + w, c] = (overlay_image[:, :, c] * mask_image) + (image[y - h: y + h, x - w: x + w, c]) * (1 - mask_image)

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈 사용
 
path = "C:\\Users\\User\\Desktop\\code\\gitFolder\\myStudy\\3_OpenCV_python\\pythonImageWorkspace\\Data"
cap = cv2.VideoCapture(path + '/face_video.mp4')

# image_right_eye = cv2.imread(path + '/right_eye.png') # 100 * 100
# image_left_eye = cv2.imread(path + '/left_eye.png') # 100 * 100
# image_nose_tip = cv2.imread(path + '/nose_tip.png') # 300 * 100

image_right_eye = cv2.imread(path + '/right.png', cv2.IMREAD_UNCHANGED) # 100 * 100
image_left_eye = cv2.imread(path + '/left.png', cv2.IMREAD_UNCHANGED) # 100 * 100
image_nose_tip = cv2.imread(path + '/nose.png', cv2.IMREAD_UNCHANGED) # 300 * 100

## model_selection
# 모델 종류
# 0 : 근거리, 1: 원거리

## min_detection_confidence
# 얼굴이라고 판단한 확률에 대한 Threshold
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 얼굴을 찾으면 그리기
        if results.detections:
            ## 6 개 포인트의 좌표 반환
            # 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection)
                # print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]

                h, w, _ = image.shape # height, width, channel
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))

                # # 양 눈에 동그라미 그리기
                # cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA)
                # cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA)
                # cv2.circle(image, nose_tip, 50, (0, 0, 255), 10, cv2.LINE_AA)

                # # image 삽입
                # image[right_eye[1] - 50 : right_eye[1] + 50, right_eye[0] - 50 : right_eye[0] + 50] = image_right_eye
                # image[left_eye[1] - 50 : left_eye[1] + 50, left_eye[0] - 50 : left_eye[0] + 50] = image_left_eye
                # image[nose_tip[1] - 50 : nose_tip[1] + 50, nose_tip[0] - 150 : nose_tip[0] + 150] = image_nose_tip
                
                # 바탕이 투명한 이미지로 편집하지 못 하여 불투명한 이미지를 삽입함
                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose_tip, 150, 50, image_nose_tip)

        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy = 0.5))
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [12]:
import cv2
path = "C:\\Users\\User\\Desktop\\code\\gitFolder\\myStudy\\3_OpenCV_python\\pythonImageWorkspace\\Data"

test = cv2.imread(path+'/right.png', cv2.IMREAD_UNCHANGED)
print(test.shape)

(100, 100, 4)
(100, 100)
